In [18]:
import pandas as pd
import numpy as np
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [19]:
import nltk
from nltk.corpus import stopwords 
from nltk.stem import PorterStemmer as PS
from nltk.tokenize import WordPunctTokenizer as WPT
import re

In [21]:
stopwords_list = stopwords.words('english')

In [22]:
def clean_text(text):
    text = re.sub(r'http\S+', '', text)
    text = re.sub(r'@\S+', '', text)
    text = re.sub(r'#\S+', '', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = text.lower()
    text = text.strip()
    text = ' '.join([word for word in text.split() if word not in stopwords_list])
    return text

In [23]:
!pip install -U sentence-transformers

In [24]:
from sentence_transformers import SentenceTransformer
enc_model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

In [25]:
train_val = pd.read_csv('train.csv')
test_val = pd.read_csv('test.csv')

In [26]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

from unidecode import unidecode
X_train_val = train['text'].apply(unidecode).to_numpy()
X_test = test['text'].apply(unidecode).to_numpy()

y_train_val = train_val.target.to_numpy()

In [27]:
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.1, random_state=42)

In [28]:
X_train = enc_model.encode(X_train)
X_val = enc_model.encode(X_val)
X_test = enc_model.encode(X_test)

In [29]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=140)
model.fit(X_train, y_train)

RandomForestClassifier(n_estimators=140)

In [31]:
import lightgbm as lgb
model = lgb.LGBMClassifier(learning_rate=0.1,random_state=50)
model.fit(X_train, y_train,eval_set=(X_val,y_val),
          eval_metric='accuracy')

[1]	valid_0's binary_logloss: 0.660657
[2]	valid_0's binary_logloss: 0.640101
[3]	valid_0's binary_logloss: 0.620156
[4]	valid_0's binary_logloss: 0.602978
[5]	valid_0's binary_logloss: 0.588768
[6]	valid_0's binary_logloss: 0.575545
[7]	valid_0's binary_logloss: 0.562111
[8]	valid_0's binary_logloss: 0.551972
[9]	valid_0's binary_logloss: 0.54307
[10]	valid_0's binary_logloss: 0.532639
[11]	valid_0's binary_logloss: 0.524958
[12]	valid_0's binary_logloss: 0.51687
[13]	valid_0's binary_logloss: 0.510189
[14]	valid_0's binary_logloss: 0.504343
[15]	valid_0's binary_logloss: 0.498209
[16]	valid_0's binary_logloss: 0.493624
[17]	valid_0's binary_logloss: 0.487509
[18]	valid_0's binary_logloss: 0.482988
[19]	valid_0's binary_logloss: 0.47897
[20]	valid_0's binary_logloss: 0.476068
[21]	valid_0's binary_logloss: 0.471369
[22]	valid_0's binary_logloss: 0.468071
[23]	valid_0's binary_logloss: 0.466052
[24]	valid_0's binary_logloss: 0.463397
[25]	valid_0's binary_logloss: 0.460376
[26]	valid_0

LGBMClassifier(random_state=50)

In [32]:
val_ans = model.predict(X_val)
val_acc = accuracy_score(y_val, val_ans) * 100

print(f"Validation accuracy is: {val_acc:.2f}%")

Validation accuracy is: 80.84%


In [33]:
test_ans = model.predict(X_test)
submission_df = pd.DataFrame({"id": test.id, "target": test_ans})

In [34]:
submission_df.to_csv("submission.csv", index=False)